In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from fastai.imports import *
np.set_printoptions(linewidth=130)

# Data exploration and preprocessing

In [6]:
creds = """{"username":"dogxyz","key":"f39bca0d1d833d63fa7dad15450de268"}"""

from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [7]:

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

compname = 'feedback-prize-english-language-learning'

if iskaggle: path = Path('../input/'+compname)
else:
    import zipfile,kaggle
    path = Path(compname)
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

df = pd.read_csv(path/'train.csv')
tst_df = pd.read_csv(path/'test.csv')
modes = df.mode().iloc[0]

100%|██████████| 2.80M/2.80M [00:00<00:00, 30.9MB/s]

In [8]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [9]:
df[:1]

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and st...",3.5,3.5,3.0,3.0,4.0,3.0


In [10]:
df.select_dtypes(include=[np.number]).sum(axis=1)

0       20.0
1       14.5
2       18.0
3       27.0
4       16.5
        ... 
3906    17.0
3907    22.5
3908    18.0
3909    26.0
3910    19.0
Length: 3911, dtype: float64

In [11]:
def proc_data(df):
    if not 'score' in df.columns:
        df['score'] = df.select_dtypes(include=[np.number]).sum(axis=1)

proc_data(df)
proc_data(tst_df)

In [12]:
df[:1]

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,score
0,0016926B079C,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and st...",3.5,3.5,3.0,3.0,4.0,3.0,20.0


In [13]:
max_scored = df.score.idxmax()
max_scored

2389

In [14]:
print('Best overall essay:')
print(df.loc[max_scored].full_text)

Best overall essay:
I agree with Michelangelo's statement as I have found through experience that it benefits me more to set high expectations and not reach my goal, rather than settling on a low goal and achieving it. When setting high goals, I find that I learn more and progress my abilties further than I do with a lower goal. This is because setting high goals requires confidence, challenges, and pride.

Firstly, hard work is required to achieve high goals, and requires confidence. Confidence involves having trust and believing that you are capable of accomplishing something. I have found that when I set high goals, I feel more confident in my abilities. With lower goals however, my confidence is lower as I settle for the easier path and do not trust myself to do better. With a higher aim, my confidence motivates me to not give up. I tend to try harder, and always believe in myself. For example, at school, I had to choose whether I wanted to try out for the varsity tennis team or re

In [15]:
# let's check for missing data
df.isna().sum(axis=0)

text_id        0
full_text      0
cohesion       0
syntax         0
vocabulary     0
phraseology    0
grammar        0
conventions    0
score          0
dtype: int64

In [16]:
list(df.columns)

['text_id',
 'full_text',
 'cohesion',
 'syntax',
 'vocabulary',
 'phraseology',
 'grammar',
 'conventions',
 'score']

In [17]:
# let's see general training set statistics
df.describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,score
count,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000
mean,3.127077,3.028254,3.235745,3.116850,3.032856,3.081053,18.621836
std,0.662542,0.644399,0.583148,0.655997,0.699841,0.671450,3.365642
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000,16.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,18.500000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,21.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,30.000000


# Let's 

In [18]:
model_name = 'microsoft/deberta-v3-small'

from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [19]:
ds

Dataset({
    features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'score'],
    num_rows: 3911
})

In [20]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
' '.join(tokenizer.tokenize("Hi! this is dog-person, the learning d0g, extrapolating the absolute beginners notebook"))

'▁Hi ! ▁this ▁is ▁dog - person , ▁the ▁learning ▁d 0 g , ▁extrapolating ▁the ▁absolute ▁beginners ▁notebook'

In [22]:
def tok_func(x): return tokenizer(x["full_text"])
tok_func(ds[0])

{'input_ids': [1, 273, 428, 272, 598, 338, 1591, 292, 1101, 288, 425, 261, 10405, 306, 13521, 286, 264, 575, 263, 350, 322, 737, 267, 262, 1066, 264, 3038, 263, 333, 343, 1245, 260, 787, 364, 2141, 1530, 349, 401, 288, 343, 669, 306, 280, 436, 282, 794, 310, 1251, 260, 306, 296, 282, 1800, 288, 425, 260, 279, 9769, 465, 265, 563, 269, 646, 1041, 260, 274, 4302, 322, 424, 5063, 290, 3261, 263, 424, 264, 290, 6928, 263, 468, 288, 290, 36738, 260, 385, 274, 428, 274, 2847, 266, 5776, 3636, 424, 468, 267, 262, 4520, 263, 274, 436, 814, 298, 334, 278, 289, 274, 468, 263, 398, 266, 10997, 260, 1060, 274, 280, 436, 286, 264, 575, 260, 275, 262, 535, 2141, 274, 295, 1929, 784, 263, 992, 425, 263, 274, 13521, 389, 264, 2148, 314, 339, 264, 1929, 260, 370, 598, 1048, 413, 10796, 416, 563, 260, 306, 814, 413, 278, 416, 306, 1873, 289, 335, 306, 4302, 322, 260, 347, 598, 333, 462, 264, 4984, 397, 260, 272, 2884, 349, 333, 2374, 262, 2444, 263, 1563, 277, 343, 4178, 326, 1138, 306, 488, 1109, 264, 

In [23]:
tok_ds = ds.map(tok_func, batched=True)

Parameter 'function'=<function tok_func at 0x7fc096635790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

In [24]:
tok_ds

Dataset({
    features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'score', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3911
})

In [25]:
row = tok_ds[0]

print(row['full_text'])
print(' '.join(map(str, row['input_ids'])))

I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.

The hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.

most students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go take a

In [26]:
tokenizer.vocab['▁students']

598

In [27]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [30]:
dds = tok_ds.train_test_split(0.20, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3128
    })
    test: Dataset({
        features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 783
    })
})

In [31]:
eval_df = tst_df

In [35]:
from transformers import TrainingArguments,Trainer
bs = 16
epochs = 4
lr = 8e-5
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

PyTorch: setting up devices


In [36]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokenizer, compute_metrics=corr_d)

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input"

In [37]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: phraseology, cohesion, vocabulary, grammar, syntax, full_text, text_id, conventions. If phraseology, cohesion, vocabulary, grammar, syntax, full_text, text_id, conventions are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3128
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 784


RuntimeError: CUDA out of memory. Tried to allocate 332.00 MiB (GPU 0; 15.74 GiB total capacity; 13.78 GiB already allocated; 333.56 MiB free; 14.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF